### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bone_Density/GSE56814'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Gene Expression Data Availability
is_gene_available = True  # The dataset involves microarray analysis, which implies gene expression data

# Variable Availability and Data Type Conversion

# Bone_Density (trait)
trait_row = 1  # Bone Density data is available under key 1
def convert_trait(value):
    if ':' in value:
        value = value.split(':')[1].strip().lower()
        if value in ['high bmd', 'low bmd']:
            return 1 if value == 'high bmd' else 0
    return None

# Age
age_row = None  # Age data is not explicitly available in the sample characteristics dictionary

# Gender
gender_row = 0  # Gender data is available under key 0
def convert_gender(value):
    if ':' in value:
        value = value.split(':')[1].strip().lower()
        if value == 'female':
            return 0  # Female as 0
        # Assuming no male data as per dataset description
    return None

# Save Metadata
save_cohort_info('GSE56814', './preprocessed/Bone_Density/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Bone_Density', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Bone_Density/trait_data/GSE56814.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine identifier and gene_symbol keys from the previewed gene annotation dictionary in STEP5
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Check the columns in gene_mapping_df to ensure 'gene_assignment' is present
print("Columns in gene_mapping_df:", gene_mapping_df.columns)

def parse_gene_symbols(gene_string):
    if pd.isna(gene_string):
        return None
    # Split and extract valid gene symbols from 'gene_assignment'
    symbols = [entry.strip().split(' // ')[1] for entry in gene_string.split(' /// ') if ' // ' in entry]
    return ' // '.join(symbols)

# Apply the parsing function to extract meaningful gene symbols
gene_mapping_df['Gene'] = gene_mapping_df[gene_symbol_key].apply(parse_gene_symbols)

# Ensure mapping is done cleanly on the necessary columns
gene_mapping_df = gene_mapping_df[[identifier_key, 'Gene']]

# 3. Apply the mapping with the 'apply_gene_mapping' function and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Bone_Density/gene_data/GSE56814.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Bone_Density')

# 4. Save the cohort information.
save_cohort_info('GSE56814', './preprocessed/Bone_Density/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Bone_Density/GSE56814.csv'
    unbiased_merged_data.to_csv(csv_path)
